<a href="https://colab.research.google.com/github/tumblingdice512/Research/blob/master/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

モデル2 シンプルなnumpyによる実装

tanh関数の利用

(逆伝播の実装がネック)

モデルについて

幅N、深さL+1(l=0,1,･･･,L)のネットワーク

各パーセプトロンはM成分を持つスピンで各成分は+1か-1を取る

第l層と第l+1層の各パーセプトロンの同一成分は全て結合→重みJ

各層の重みJはΣJ**2 = Nと正規化されている

第l層のスピンの値に重みをかけた値に対して符号関数をかませたものが第l+1層の値(0に対して+1か-1にする)

初期条件としては、入力層と出力層のスピンの値を与える

入力層のスピンの値と、初期の重みの条件から、出力層を予想して、損失関数を小さくするために重みを更新する

In [1]:
import jax
import numpy as np
import random

In [2]:
N = 20
L = 5

In [3]:
a = [1, -1]
b = random.choice(a)

print(b)

1


In [4]:
#S0 = np.array([random.choice([1,-1]) for i in range(N)])
#SL = np.array([random.choice([1,-1]) for i in range(N)])

#print(S0)
#print("S0の要素数は",len(S0))

#print(SL)
#print("SLの要素数は",len(SL))

#初期条件として与えるスピンを生成
#各パーセプトロンのスピンの成分数を増やすときは、SOとSLを行列の形式にすればよい。

In [5]:
S0 = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]
               ,[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]
               ,[random.choice([1,-1]) for i in range(N)]])

SL = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]
               ,[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]
               ,[random.choice([1,-1]) for i in range(N)]])

M = len(S0)

print(S0) #同じ配列内は、スピンの第μ成分がN個並んでいる。各パーセプトロンに直すには、転置を取る必要あり
print(M) #Mに相当

print(SL)




[[-1  1  1  1  1  1 -1 -1  1 -1  1 -1  1  1 -1 -1  1  1 -1 -1]
 [-1  1  1  1 -1  1  1  1 -1  1  1 -1  1  1  1 -1  1  1 -1 -1]
 [-1 -1 -1  1 -1  1 -1 -1  1  1  1  1 -1  1  1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1  1  1 -1  1  1 -1 -1 -1 -1  1  1 -1 -1  1]
 [ 1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1]
 [ 1 -1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1  1 -1]
 [-1  1 -1 -1 -1 -1  1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1  1  1]]
7
[[ 1  1 -1  1  1 -1  1 -1  1 -1 -1 -1  1 -1  1 -1  1  1 -1 -1]
 [ 1  1  1 -1 -1  1 -1  1  1  1  1 -1  1  1 -1  1  1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1 -1  1  1  1 -1 -1  1 -1 -1 -1  1 -1  1]
 [ 1 -1 -1 -1 -1  1  1  1  1  1  1  1 -1  1 -1  1 -1  1  1 -1]
 [-1 -1 -1  1 -1 -1  1 -1 -1  1  1  1 -1 -1  1  1  1  1  1  1]
 [-1  1 -1 -1  1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1 -1]
 [-1 -1  1  1 -1  1  1  1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1 -1]]


In [6]:
def spin_overlap(x,y):
  for i in range(N) :

    spin_overlap = np.dot(x.T[i],y.T[i])
    return np.abs(spin_overlap/(M*N))

spin_overlap(S0,SL)

0.02142857142857143

In [7]:
def MSE(t, y):
    mse = np.mean(np.sum(np.square(t-y),axis =1),axis = 0)
    return mse



In [8]:
A = np.array([[3,5,1],[4,12,1]])
print(A**2)
A_norm = (np.sum(np.square(A),axis=0))**(1/2)

A_normalized = A / A_norm
A_normalized_2 = A / A_norm * (N**(1/2))

print(A_norm)

print(A_normalized)
print(A_normalized_2)


[[  9  25   1]
 [ 16 144   1]]
[ 5.         13.          1.41421356]
[[0.6        0.38461538 0.70710678]
 [0.8        0.92307692 0.70710678]]
[[2.68328157 1.72005229 3.16227766]
 [3.57770876 4.1281255  3.16227766]]


In [9]:
def weight_norm(x):
    x_norm = (np.sum(np.square(x),axis=0)**(1/2))
    return x_norm

In [10]:
class Network1():
  #ネットワークを定義する
  #構成[入力層, 第1層, 第2層, 第3層, 第4層, 第5層]
  #全結合

  def __init__(self):
    #重みの定義
    self.w1 = np.random.randn(N,N)
    self.w2 = np.random.randn(N,N)
    self.w3 = np.random.randn(N,N)
    self.w4 = np.random.randn(N,N)
    self.w5 = np.random.randn(N,N)

    #重みの正規化
    self.J1 = self.w1 / (weight_norm(self.w1)) * (N**(1/2))
    self.J2 = self.w2 / (weight_norm(self.w2)) * (N**(1/2))
    self.J3 = self.w3 / (weight_norm(self.w3)) * (N**(1/2))
    self.J4 = self.w4 / (weight_norm(self.w4)) * (N**(1/2))
    self.J5 = self.w5 / (weight_norm(self.w5)) * (N**(1/2))

  def forward(self, x):

    self.layer0_1 = x
    self.layer1_1 = np.tanh((np.dot(self.layer0_1, self.J1))+1e-7)
    self.layer2_1 = np.tanh((np.dot(self.layer1_1, self.J2))+1e-7)
    self.layer3_1 = np.tanh((np.dot(self.layer2_1, self.J3))+1e-7)
    self.layer4_1 = np.tanh((np.dot(self.layer3_1, self.J4))+1e-7)
    self.out_1 = np.tanh((np.dot(self.layer4_1, self.J5))+1e-7)
    return self.out_1
    

  def backward(self, t, y):
    #誤差逆伝播
    delta5 = -2*(t-y)
    delta4 = (1-(np.tanh(np.dot(self.layer4_1, self.J5)))**2)*np.dot(delta5,self.J5.T)
    delta3 = (1-(np.tanh(np.dot(self.layer3_1, self.J4)))**2)*np.dot(delta4,self.J4.T) 
    delta2 = (1-(np.tanh(np.dot(self.layer2_1, self.J3)))**2)*np.dot(delta3,self.J3.T)
    delta1 = (1-(np.tanh(np.dot(self.layer1_1, self.J2)))**2)*np.dot(delta2,self.J2.T)

    #重みの勾配
    self.dedJ5 = np.dot(self.layer4_1.T, delta5) / delta5.shape[0]
    self.dedJ4 = (1-(np.tanh(np.dot(self.layer4_1, self.J5)))**2)*np.dot(self.layer3_1.T, delta4) / delta4.shape[0]
    self.dedJ3 = (1-(np.tanh(np.dot(self.layer3_1, self.J4)))**2)*np.dot(self.layer2_1.T, delta3) / delta3.shape[0]
    self.dedJ2 = (1-(np.tanh(np.dot(self.layer2_1, self.J3)))**2)*np.dot(self.layer1_1.T, delta2) / delta2.shape[0]
    self.dedJ1 = (1-(np.tanh(np.dot(self.layer1_1, self.J2)))**2)*np.dot(self.layer0_1.T, delta1) / delta1.shape[0]

  def optimize_GradientDecent(self, lr):
    #重みの更新    
    self.J1 -= lr * self.dedJ1
    self.J2 -= lr * self.dedJ2
    self.J3 -= lr * self.dedJ3
    self.J4 -= lr * self.dedJ4
    self.J5 -= lr * self.dedJ5

    #重みの正規化
    self.J1 = self.J1 / (weight_norm(self.J1)) * (N**(1/2))
    self.J2 = self.J2 / (weight_norm(self.J2)) * (N**(1/2))
    self.J3 = self.J3 / (weight_norm(self.J3)) * (N**(1/2))
    self.J4 = self.J4 / (weight_norm(self.J4)) * (N**(1/2))
    self.J5 = self.J5 / (weight_norm(self.J5)) * (N**(1/2))



In [11]:
class Network2():
  #ネットワークを定義する
  #構成[入力層, 第1層, 第2層, 第3層, 第4層, 第5層]
  #全結合

  def __init__(self):
    #重みの定義
    self.w1 = np.random.randn(N,N)
    self.w2 = np.random.randn(N,N)
    self.w3 = np.random.randn(N,N)
    self.w4 = np.random.randn(N,N)
    self.w5 = np.random.randn(N,N)

    #重みの正規化
    self.J1 = self.w1 / (weight_norm(self.w1)) * (N**(1/2))
    self.J2 = self.w2 / (weight_norm(self.w2)) * (N**(1/2))
    self.J3 = self.w3 / (weight_norm(self.w3)) * (N**(1/2))
    self.J4 = self.w4 / (weight_norm(self.w4)) * (N**(1/2))
    self.J5 = self.w5 / (weight_norm(self.w5)) * (N**(1/2))

  def forward(self, x):

    self.layer0_2 = x
    self.layer1_2 = np.tanh((np.dot(self.layer0_2, self.J1))+1e-7)
    self.layer2_2 = np.tanh((np.dot(self.layer1_2, self.J2))+1e-7)
    self.layer3_2 = np.tanh((np.dot(self.layer2_2, self.J3))+1e-7)
    self.layer4_2= np.tanh((np.dot(self.layer3_2, self.J4))+1e-7)
    self.out_2 = np.tanh((np.dot(self.layer4_2, self.J5))+1e-7)
    return self.out_2
    

  def backward(self, t, y):
    #誤差逆伝播
    delta5 = -2*(t-y)
    delta4 = (1-(np.tanh(np.dot(self.layer4_2, self.J5)))**2)*np.dot(delta5,self.J5.T)
    delta3 = (1-(np.tanh(np.dot(self.layer3_2, self.J4)))**2)*np.dot(delta4,self.J4.T) 
    delta2 = (1-(np.tanh(np.dot(self.layer2_2, self.J3)))**2)*np.dot(delta3,self.J3.T)
    delta1 = (1-(np.tanh(np.dot(self.layer1_2, self.J2)))**2)*np.dot(delta2,self.J2.T)

    #重みの勾配
    self.dedJ5 = np.dot(self.layer4_2.T, delta5) / delta5.shape[0]
    self.dedJ4 = (1-(np.tanh(np.dot(self.layer4_2, self.J5)))**2)*np.dot(self.layer3_2.T, delta4) / delta4.shape[0]
    self.dedJ3 = (1-(np.tanh(np.dot(self.layer3_2, self.J4)))**2)*np.dot(self.layer2_2.T, delta3) / delta3.shape[0]
    self.dedJ2 = (1-(np.tanh(np.dot(self.layer2_2, self.J3)))**2)*np.dot(self.layer1_2.T, delta2) / delta2.shape[0]
    self.dedJ1 = (1-(np.tanh(np.dot(self.layer1_2, self.J2)))**2)*np.dot(self.layer0_2.T, delta1) / delta1.shape[0]


  def optimize_GradientDecent(self, lr):
    #重みの更新    
    self.J1 -= lr * self.dedJ1
    self.J2 -= lr * self.dedJ2
    self.J3 -= lr * self.dedJ3
    self.J4 -= lr * self.dedJ4
    self.J5 -= lr * self.dedJ5

    #重みの正規化
    self.J1 = self.J1 / (weight_norm(self.J1)) * (N**(1/2))
    self.J2 = self.J2 / (weight_norm(self.J2)) * (N**(1/2))
    self.J3 = self.J3 / (weight_norm(self.J3)) * (N**(1/2))
    self.J4 = self.J4 / (weight_norm(self.J4)) * (N**(1/2))
    self.J5 = self.J5 / (weight_norm(self.J5)) * (N**(1/2))

    

In [12]:
model_1 = Network1()
model_2 = Network2()

# 学習率
lr = 0.01
# 学習エポック数
n_epoch = 1000

# n_epoch繰り返す
for n in range(n_epoch):
    y_1 = model_1.forward(S0)
    y_2 = model_2.forward(S0)

    train_loss_1 = MSE(SL, y_1)
    train_loss_2 = MSE(SL, y_2)

    model_1.backward(SL, y_1)
    model_2.backward(SL, y_2)

    model_1.optimize_GradientDecent(lr)
    model_2.optimize_GradientDecent(lr)

    #spin_overlap_0 = np.sum()

    print('EPOCH ', n + 1, ' | TRAIN LOSS_1 ',train_loss_1, '  TRAIN LOSS_2 ',train_loss_2)

ValueError: ignored

In [ ]:
print(y_1)
print(y_2)
print(SL)

[[ 1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.
   1. -1.]
 [ 1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.
   1. -1.]
 [-1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.
  -1.  1.]
 [-1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.
  -1.  1.]
 [ 1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.
   1. -1.]
 [ 1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.
   1. -1.]
 [-1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.
  -1.  1.]]
[[-1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.
  -1. -1.]
 [-1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.
  -1. -1.]
 [ 1. -1. -1. -1.  1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1. -1. -1. -1.
   1. -1.]
 [-1.  1. -1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1.
  -1.  1.]
 [ 1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1